In [1]:
import numpy as np
import pandas as pd

# Data Preprocessing

In [2]:
# prepare internal FAQ dataset
df_faq = pd.read_excel('faq-data/df_undergrad_faq.xlsx')

df_temp = pd.read_csv('faq-data/df_6901_faq.csv', index_col=0)
df_temp['Type'] = 'BSc 6901'
df_faq = df_faq.append(df_temp)

df_temp = pd.read_excel('faq-data/df_basc_faq.xlsx')
df_temp['Type'] = 'BASc'
df_faq = df_faq.append(df_temp)

df_temp = pd.read_csv('faq-data/df_aao_faq.csv', index_col=0)
df_temp['Type'] = 'AAO'
df_faq = df_faq.append(df_temp)

df_faq = df_faq.reset_index(drop=True)

# include the type of the question into both 'Question' and 'Answer'
df_faq['Question'] = '('+ df_faq['Type'] + ') ' + df_faq['Question']
df_faq['Answer'] = '('+ df_faq['Type'] + ') ' + df_faq['Answer']

In [3]:
# save preproecessed faq dataset
df_faq.to_csv('faq-data/df_faq.csv')

In [4]:
df_faq.sample(10)

,Question,Answer,Type
45,(BSc 6901) If I do the Major-Minor instead of ...,(BSc 6901) For those who intend to develop a c...,BSc 6901
83,(AAO) Can I change my FAA?,"(AAO) Under normal circumstances, FAA cannot b...",AAO
63,(BSc 6901) Who is eligible to apply for the sc...,(BSc 6901) Candidates who hold a recognised fu...,BSc 6901
76,(BASc) Can I participate in the exchange progr...,(BASc) Though it is not a compulsory graduatio...,BASc
81,(AAO) How can I find my Faculty Academic Advis...,(AAO) You can check who your FAA is via the we...,AAO
38,"(BSc 6901) If that situation arises, would tha...",(BSc 6901) Absolutely not. The Faculty will ad...,BSc 6901
57,"(BSc 6901) If I get 5 or above in DSE English,...","(BSc 6901) Yes, the first English course will ...",BSc 6901
51,(BSc 6901) Do I need to put the Programme in B...,(BSc 6901) Over 98% of students admitted in th...,BSc 6901
87,(AAO) I am interested in the programmes of HKU...,"(AAO) For matters related to admission, please...",AAO
167,(AAO) What fees do I have to pay?,(AAO) You can find detailed information on “Un...,AAO


# Helper Functions

In [5]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk import word_tokenize
from nltk.corpus import stopwords
import string

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kackie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kackie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
class Sentence:
    def __init__(self, sentence):
        self.sentence = sentence

    # METHOD: preprocess the sentence
    # return: list of tokenized words
    def preprocess(self, with_stopwords=False):
        text = self.sentence

        # convert text to lower case
        text = text.lower()

        # tokenize text into list of words
        words = word_tokenize(text)

        # remove punctuation
        words = [word for word in words if word.isalpha()]

        if with_stopwords == False:
            # remove stopwords
            stop_words = set(stopwords.words('english'))
            words = [word for word in words if not word in stop_words]

        return words
        
    # METHOD: get word embeddings based on specific model
    # return: matrix word embeddings
    def get_vector(self, model, with_stopwords=False):
        return np.sum(np.array([model[i] for i in self.preprocess(with_stopwords=with_stopwords)]), axis=0)

In [7]:
from scipy import spatial

# HELPER FUNCTION: get similarities of a specific query against all questions in our FAQ database
# return: dataframe with new column 'Similarity'
def get_similarities(query, df, model, with_stopwords=False):
    similarities = []
    query = Sentence(query)
    for q in df['Question']:
        s = Sentence(q)
        try:
            similarity = 1 - spatial.distance.cosine(
                query.get_vector(model, with_stopwords), 
                s.get_vector(model, with_stopwords)
            )
            similarities.append(similarity)
        except:
            similarities.append(0)
            continue
    df['Similarity'] = similarities
    
    df = df.sort_values(by='Similarity', ascending=False)

    return df

---
# Models

In [8]:
# define any query
QUERY = 'I wanna know what the application deadline is.'

## Model 1: Word2Vec

In [9]:
import gensim.downloader as api

# define word2vec model
model_w2v = api.load('word2vec-google-news-300')

**WITHOUT stopwords**

In [10]:
# get cosine similarity scores for every entry in our FAQ database
# WITHOUT stopwords
get_similarities(QUERY, df_faq, model_w2v, with_stopwords=False).to_csv('model-sample-results/w2v-without-stopwords.csv')

In [30]:
# try another query
get_similarities(
    'I am an international student. I wanna know how much the programme costs.', 
    df_faq, model_w2v, with_stopwords=False)

,Question,Answer,Type,Similarity
0,(HKDSE) How do I apply to HKU through JUPAS sc...,(HKDSE) We welcome your application to HKU thr...,HKDSE,0
120,(AAO) How do I know if I have fulfilled all th...,(AAO) It is your responsibility to check the c...,AAO,0
112,(AAO) Is it possible to double major or double...,"(AAO) Yes, as long as you have enough credits ...",AAO,0
113,(AAO) How many majors/ minors I can take to th...,(AAO) There are no rigid regulations in this r...,AAO,0
114,(AAO) Can I change my major/ minor later? How ...,(AAO) It depends on the setup which may vary f...,AAO,0
...,...,...,...,...
60,(BSc 6901) Other universities are also adoptin...,(BSc 6901) HKU Science is the first university...,BSc 6901,0
61,(BSc 6901) What is Young Scientist Scheme (YSS)?,(BSc 6901) Enrolment in our flagship Summer Re...,BSc 6901,0
62,(BSc 6901) Who is entitled to join YSS?,(BSc 6901) have a second chance to enrol in YS...,BSc 6901,0
63,(BSc 6901) Who is eligible to apply for the sc...,(BSc 6901) Candidates who hold a recognised fu...,BSc 6901,0


**WITH stopwords**

In [12]:
# get cosine similarity scores for every entry in our FAQ database
# WITH stopwords
get_similarities(QUERY, df_faq, model_w2v, with_stopwords=True).to_csv('model-sample-results/w2v-with-stopwords.csv')

In [13]:
# save model
model_w2v.save('models/w2v.model')

## Model 2: GloVe

In [14]:
import gensim.downloader as api

# define GloVe model
model_glove = api.load('glove-wiki-gigaword-300')

**WITHOUT stopwords**

In [15]:
# get cosine similarity scores for every entry in our FAQ database
# WITHOUT stopwords
get_similarities(QUERY, df_faq, model_glove, with_stopwords=False).to_csv('model-sample-results/glove-without-stopwords.csv')

In [46]:
# try another query
get_similarities(
    'I am an HKDSE student. I wanna know how much the programme costs.', 
    df_faq, model_glove, with_stopwords=False)

,Question,Answer,Type,Similarity
0,(HKDSE) How do I apply to HKU through JUPAS sc...,(HKDSE) We welcome your application to HKU thr...,HKDSE,0
120,(AAO) How do I know if I have fulfilled all th...,(AAO) It is your responsibility to check the c...,AAO,0
112,(AAO) Is it possible to double major or double...,"(AAO) Yes, as long as you have enough credits ...",AAO,0
113,(AAO) How many majors/ minors I can take to th...,(AAO) There are no rigid regulations in this r...,AAO,0
114,(AAO) Can I change my major/ minor later? How ...,(AAO) It depends on the setup which may vary f...,AAO,0
...,...,...,...,...
60,(BSc 6901) Other universities are also adoptin...,(BSc 6901) HKU Science is the first university...,BSc 6901,0
61,(BSc 6901) What is Young Scientist Scheme (YSS)?,(BSc 6901) Enrolment in our flagship Summer Re...,BSc 6901,0
62,(BSc 6901) Who is entitled to join YSS?,(BSc 6901) have a second chance to enrol in YS...,BSc 6901,0
63,(BSc 6901) Who is eligible to apply for the sc...,(BSc 6901) Candidates who hold a recognised fu...,BSc 6901,0


**WITH stopwords**

In [17]:
# get cosine similarity scores for every entry in our FAQ database
# WITH stopwords
get_similarities(QUERY, df_faq, model_glove, with_stopwords=True).to_csv('model-sample-results/glove-with-stopwords.csv')

In [18]:
# save model
model_glove.save('models/glove.model')

## Model 3: FastText

In [19]:
import gensim.downloader as api

# define FastText model
model_fasttext = api.load('fasttext-wiki-news-subwords-300')

In [20]:
# get cosine similarity scores for every entry in our FAQ database
# WITHOUT stopwords
get_similarities(QUERY, df_faq, model_fasttext, with_stopwords=False).to_csv('model-sample-results/fasttext-without-stopwords.csv')

In [52]:
# try another query
get_similarities(
    'I am a HKDSE student. I wanna know how much the programme costs.', df_faq, model_fasttext, with_stopwords=False)

,Question,Answer,Type,Similarity
0,(HKDSE) How do I apply to HKU through JUPAS sc...,(HKDSE) We welcome your application to HKU thr...,HKDSE,0
120,(AAO) How do I know if I have fulfilled all th...,(AAO) It is your responsibility to check the c...,AAO,0
112,(AAO) Is it possible to double major or double...,"(AAO) Yes, as long as you have enough credits ...",AAO,0
113,(AAO) How many majors/ minors I can take to th...,(AAO) There are no rigid regulations in this r...,AAO,0
114,(AAO) Can I change my major/ minor later? How ...,(AAO) It depends on the setup which may vary f...,AAO,0
...,...,...,...,...
60,(BSc 6901) Other universities are also adoptin...,(BSc 6901) HKU Science is the first university...,BSc 6901,0
61,(BSc 6901) What is Young Scientist Scheme (YSS)?,(BSc 6901) Enrolment in our flagship Summer Re...,BSc 6901,0
62,(BSc 6901) Who is entitled to join YSS?,(BSc 6901) have a second chance to enrol in YS...,BSc 6901,0
63,(BSc 6901) Who is eligible to apply for the sc...,(BSc 6901) Candidates who hold a recognised fu...,BSc 6901,0
